# Personalized Movie Recommendations
This notebook shows how to build a personalized movie recommendation model with ThirdAI's Universal Deep Transformer (UDT) model, our all-purpose classifier for tabular datasets. In this demo, we will train and evaluate the model on the Movielens 1M dataset, but you can easily replace this with your own dataset.

You can immediately run a version of this notebook in your browser on Google Colab at the following link:

https://githubtocolab.com/ThirdAILabs/Demos/blob/main/PersonalizedMovieRecommendations.ipynb

This notebook uses an activation key that will only work with this demo. If you want to try us out on your own dataset, you can obtain a free trial license at the following link: https://www.thirdai.com/try-bolt/

In [ ]:
!pip3 install thirdai --upgrade

import thirdai
thirdai.licensing.activate("Y9MT-TV7T-4JTP-L4XH-PWYC-4KEF-VX93-3HV7")

# Dataset Download
We will use the demos module in the thirdai package to download the Movielens 1M dataset. You can replace this step and the next step with a download method and a UDT initialization step that is specific to your dataset.

In [ ]:
from thirdai.demos import download_movielens

train_filename, test_filename, inference_batch, index_batch = download_movielens()

# UDT Initialization
We can now create a UDT model by passing in the types of each column in the dataset and the target column we want to be able to predict.

For this demo, we additionally want to use "temporal context" to make predictions. Adding temporal context requires a single bolt.types.date() column to use to track the timestamp of training data. We pass in a dictionary called temporal_tracking_relationships that tells UDT we want to track movies over time for each user. This allows UDT to make better predictions for the target column by creating temporal features that take into account the historical relationship between users and movies.

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "userId": bolt.types.categorical(),
        "movieTitle": bolt.types.categorical(),
        "timestamp": bolt.types.date(),
    },
    temporal_tracking_relationships={
        "userId": ["movieTitle"]
    },
    target="movieTitle",
    n_target_classes=3706,
)

# Training
We can now train our UDT model with just one line! Feel free to customize the number of epochs and the learning rate; we have chosen values that give good convergence.

In [ ]:
model.train(train_filename, epochs=3, learning_rate=0.001, metrics=["recall@10"]);

# Evaluation
Evaluating the performance of the UDT model is also just one line!

In [105]:
model.evaluate(test_filename, metrics=["recall@1", "recall@10", "recall@100"]);

loading data | source './movielens_test.csv' | vectors 100021 | batches 49 | time 1s | complete

evaluate | epoch 3 | train_steps 1320 | {Recall@1: 0.0237, Recall@10: 0.132, Recall@100: 0.426} | eval_batches 49 | time 25076ms



# Saving and Loading
Saving and loading a trained UDT model to disk is also extremely straight forward.

In [106]:
save_location = "personalized_movie_recommendation.model"
# Saving
model.save(save_location)

In [107]:
# Loading
model = bolt.UniversalDeepTransformer.load(save_location)

# Let's recommend!

In [112]:
model.reset_temporal_trackers()

In [124]:
import numpy as np

prediction = model.predict({'userId': '6046', 'timestamp': '2023-01-12'})
prediction_title = model.class_name(np.argmax(prediction))

print("Recommendation:", prediction_title)

Recommendation: Godfather: Part II The (1974)


In [123]:
model.index({'userId': '6046', 'timestamp': '2023-01-12', 'movieTitle': 'Godfather The (1972)'})

In [116]:
model.evaluate(train_filename);
model.evaluate(test_filename);

loading data | source './movielens_train.csv' | vectors 900188 | batches 440 | time 10s | complete

evaluate | epoch 3 | train_steps 1320 | {} | eval_batches 440 | time 90848ms

loading data | source './movielens_test.csv' | vectors 100021 | batches 49 | time 1s | complete

evaluate | epoch 3 | train_steps 1320 | {} | eval_batches 49 | time 11548ms

